In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf

In [3]:
# os.environ["TF_KERAS"]='1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


# Data

In [9]:
data  = pd.read_pickle('../data/cleaned_df.pkl')
img_path = '../data/Flipkart/images/'
df = data[['product_name', 'image', 'main_category', 'category_code']]

df['image'] = img_path + df['image']

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1050 non-null   object
 1   image          1050 non-null   object
 2   main_category  1050 non-null   object
 3   category_code  1050 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 32.9+ KB


/var/folders/4f/ml64c07d1zsg_jvvydhyltjr0000gn/T/ipykernel_54288/2050419963.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image'] = img_path + df['image']


## Prepare train and test sets

In [11]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df['image']
y = df['category_code']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)